In [1]:
"""
Ref: https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

Technique: Cross Entropy Loss

ส่วนที่ต้องพัฒนาเพิ่มเติม
1. แก้เรื่อง Overfit
2. ทำ Silmilarization เพื่อจัดการกับคำที่ไม่มีใน Dataset
3. เพิ่มจำนวน Dataset ให้มากกว่านี้
"""

In [ ]:
#Import lib

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import time

import torch
from torchtext import data, datasets, vocab
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from pythainlp.tokenize import word_tokenize

In [2]:
#ไฟล์ Dataset

word_file = pd.read_csv("รวมคำ.csv", header = None)

In [3]:
# ใน Dataset ประกอบด้วย 2 ส่วน คือคำหรือประโยค กับ Label โดย Label จะแบ่งได้ดังนี้

"""
1 = Positive
2 = High Positive
3 = Natural
4 = Negative
5 = High Negative
"""

word_file.head()

,0,1
0,ดี,1
1,ขอบคุณ,1
2,สู้เสมอ,1
3,ผ่าน,1
4,โคตรสนุก,1


In [4]:
#แปลง DataFrame ให้เป็น List เพื่อเอาไปทำ Tokenize

word_list = word_file.values.tolist()

print(word_list[0])

['ดี', 1]


In [5]:
# ทำ Tokenize ด้วย Pythainlp

new_word_list = []

for a_list in word_list:
    new_list = word_tokenize(a_list[0])
    label_list = [new_list, a_list[1]]
    new_word_list.append(label_list)

print(new_word_list)

, [['สาน', 'ฝัน', 'ให้', 'จริง'], 1], [['อาสา'], 1], [['อยาก', 'กอด'], 1], [['เตือนสติ'], 1], [['สันติสุข'], 1], [['สำนึก'], 1], [['เลือก', 'ตามใจ'], 1], [['โค', 'รต', 'น่ารัก'], 1], [['มหัศจรรย์'], 1], [['สิ่ง', 'พิเศษ'], 1], [['วิเศษ'], 1], [['ผ่อนคลาย'], 1], [['ยอมรับ'], 1], [['เจริญรุ่งเรือง'], 1], [['เสียสละ'], 1], [['ไถ่โทษ'], 1], [['ไถ่บาป'], 1], [['อบอวล'], 1], [['สะอาด'], 1], [['เอาใจใส่'], 1], [['ภาวนา'], 1], [['ฟุ้งเฟื่อง'], 1], [['เกียรติคุณ'], 1], [['ด้าน', 'บวก'], 1], [['ปลื้ม', 'ดีใจ'], 1], [['คึกคัก'], 1], [['เจตนา', 'ดี'], 1], [['จิต', 'แจ่มใส'], 1], [['เปรี้ยงปร้าง'], 1], [['ฮิต'], 1], [['ที่', 'นิยม'], 1], [['ปลอบใจ'], 1], [['ปรองดอง'], 1], [['ไมตรี'], 1], [['มารยาท'], 1], [['รอบคอบ'], 1], [['บริสุทธิ์ใจ'], 1], [['เคารพนับถือ'], 1], [['วุฒิภาวะ'], 1], [['สุขุม'], 1], [['ฟื้นฟู'], 1], [['ล้ำสมัย'], 1], [['ดี', 'ที่สุด'], 1], [['ทันเวลา', 'พอดี'], 1], [['สุจริต'], 1], [['น่าเชื่อถือ'], 1], [['เท่ห์'], 1], [['ไว้วางใจ'], 1], [['เสน่ห์'], 1], [['ทรงพลัง'], 1], [['ฮัก'], 

In [6]:
# เราจะแปลง คำให้เป็น Tensor จึงต้องเอาคำซ้ำออก โดยเปลี่ยนให้เป็น Set

word_set = set()

for a_list in new_word_list:
    word_set.update(a_list[0])
        
print(word_set)

{'คง', 'บริจาค', 'แล้ง', 'บีบบังคับ', 'ฮิต', 'ดีใจ', 'งอกงาม', 'คำสอน', 'ถูกใจ', 'เป็นกลาง', 'ยักษ์', 'ครื้น', 'เเล้ว', 'ระแวง', 'โดย', 'เสรี', 'ที่ปรึกษา', 'น่านับถือ', 'ไข้หวัด', 'ยอมแพ้', 'หรา', 'ชื่นใจ', 'โสโครก', 'ความปรารถนา', 'โม', 'ถึงตาย', 'เลือด', 'แตกแยก', 'ๆๆ', 'เธอ', 'เหลือ', 'ลืมเลือน', 'คน', 'เสียความรู้สึก', 'บรรยากาศ', 'คุณธรรม', 'ท้อแท้', 'ผีหลอก', 'ผู้ชาย', 'ชิบหาย', 'พยายาม', ' ', 'กระทืบ', 'โรค', 'ว่าง', 'ตอแหล', 'ป่า', 'ยัง', 'ริดรอน', 'เบอร์', 'ย้าย', 'อาฆาต', 'กรุ่ม', 'โว้ย', 'แสนเข็ญ', 'ที่จะ', 'พราก', 'ดาว', 'กพอเเล้ว', 'ทางออก', 'โจมตี', 'เปิด', 'วิบัติ', 'วัน', 'มี', 'คล่องตัว', 'ขาดดุล', 'เย็นชา', 'เพิ่ง', 'น่าเชื่อถือ', 'สว่าง', 'ทางใจ', 'แต่', 'ใส้', 'การ', 'แกล้ง', 'ลด', 'เคย', 'เมือง', 'ล่าช้า', 'ยืดหยุ่น', 'ผัก', 'ความเสี่ยง', 'รักษ์', 'เจ็บปวด', 'เหินห่าง', 'มือ', 'หลบ', 'มักจะ', 'เจตนา', 'กังวล', 'ปฎิบัติ', 'ด', 'ห่วง', 'เดา', 'ภาพ', 'ลอง', 'กลาง', 'เก่ง', 'เข้มแข็ง', 'การตาม', 'ผิดเพี้ยน', 'เนี่ย', 'เขา', 'นำ', 'ล้ม', 'หือ', 'แก่ง', 'ลืม', 'งาน', 'ว

In [7]:
#Set เอาคำซ้อออกหมดแล้ว เอากลับไปเป็น List อีกครั้ง

word_set_to_list = list(word_set)

print(word_set_to_list)

['คง', 'บริจาค', 'แล้ง', 'บีบบังคับ', 'ฮิต', 'ดีใจ', 'งอกงาม', 'คำสอน', 'ถูกใจ', 'เป็นกลาง', 'ยักษ์', 'ครื้น', 'เเล้ว', 'ระแวง', 'โดย', 'เสรี', 'ที่ปรึกษา', 'น่านับถือ', 'ไข้หวัด', 'ยอมแพ้', 'หรา', 'ชื่นใจ', 'โสโครก', 'ความปรารถนา', 'โม', 'ถึงตาย', 'เลือด', 'แตกแยก', 'ๆๆ', 'เธอ', 'เหลือ', 'ลืมเลือน', 'คน', 'เสียความรู้สึก', 'บรรยากาศ', 'คุณธรรม', 'ท้อแท้', 'ผีหลอก', 'ผู้ชาย', 'ชิบหาย', 'พยายาม', ' ', 'กระทืบ', 'โรค', 'ว่าง', 'ตอแหล', 'ป่า', 'ยัง', 'ริดรอน', 'เบอร์', 'ย้าย', 'อาฆาต', 'กรุ่ม', 'โว้ย', 'แสนเข็ญ', 'ที่จะ', 'พราก', 'ดาว', 'กพอเเล้ว', 'ทางออก', 'โจมตี', 'เปิด', 'วิบัติ', 'วัน', 'มี', 'คล่องตัว', 'ขาดดุล', 'เย็นชา', 'เพิ่ง', 'น่าเชื่อถือ', 'สว่าง', 'ทางใจ', 'แต่', 'ใส้', 'การ', 'แกล้ง', 'ลด', 'เคย', 'เมือง', 'ล่าช้า', 'ยืดหยุ่น', 'ผัก', 'ความเสี่ยง', 'รักษ์', 'เจ็บปวด', 'เหินห่าง', 'มือ', 'หลบ', 'มักจะ', 'เจตนา', 'กังวล', 'ปฎิบัติ', 'ด', 'ห่วง', 'เดา', 'ภาพ', 'ลอง', 'กลาง', 'เก่ง', 'เข้มแข็ง', 'การตาม', 'ผิดเพี้ยน', 'เนี่ย', 'เขา', 'นำ', 'ล้ม', 'หือ', 'แก่ง', 'ลืม', 'งาน', 'ว

In [8]:
#ทำ Set สำหรับตัว Label

label_set = set()

for a_list in new_word_list:
    if a_list[1] in label_set:
        pass
    else:
        label_set.add(a_list[1])


print(label_set)

{1, 2, 3, 4, 5}


In [9]:
print(len(word_set))
print(len(label_set))

1457
5


In [10]:
#อันนี้ลองแปลงคำเป็นตัวเลขดูเฉยๆ
#หลักการที่จะทำคือ แปลงคำเป็นตัวเลขตั้งแต่ 0 ขึ้นไปแล้วเก็บไว้ใน Numpy Array แล้วเอา Array มาแปลงเป็น Tensor อีกทีนึง

sentence_idx = np.linspace(0,len(word_set_to_list),len(word_set_to_list),False)

sentence_idx[0]

0.0

In [11]:
#แปลงคำเป็นตัวเลข โดยแต่ละคำจะมีเลขของตัวเองกำกับไว้

a = 0
link_list_1 = []

while a < len(sentence_idx):
    inner_list = []
    inner_list.append(word_set_to_list[a])
    inner_list.append(sentence_idx[a])
    link_list_1.append(inner_list)
    a += 1

print(link_list_1)

', 400.0], ['แคร์', 401.0], ['สนุก', 402.0], ['รา', 403.0], ['ทอดทิ้ง', 404.0], ['หน้ากาก', 405.0], ['ความชอบ', 406.0], ['สลาย', 407.0], ['บันได', 408.0], ['ท้าทาย', 409.0], ['สม', 410.0], ['เถียง', 411.0], ['กดขี่', 412.0], ['เพ', 413.0], ['ความสูญเสีย', 414.0], ['รวม', 415.0], ['โคตร', 416.0], ['แล้ว', 417.0], ['สังเวย', 418.0], ['หมั่น', 419.0], ['กบฏ', 420.0], ['ถูกต้อง', 421.0], ['กวน', 422.0], ['ที', 423.0], ['เสมอ', 424.0], ['ขัดขืน', 425.0], ['น้ำป่า', 426.0], ['กล้าหาญ', 427.0], ['ับ', 428.0], ['อพยพ', 429.0], ['จม', 430.0], ['ฝันดี', 431.0], ['รบกวน', 432.0], ['หาไม่', 433.0], ['มรณะ', 434.0], ['ตา', 435.0], ['อีกไม่นาน', 436.0], ['สา', 437.0], ['ป่าไม้', 438.0], ['ผู้อื่น', 439.0], ['คุกคาม', 440.0], ['อุทิศ', 441.0], ['แสบท้อง', 442.0], ['อากาศ', 443.0], ['ปรารถนา', 444.0], ['แล', 445.0], ['เหน็บแนม', 446.0], ['ประหยัด', 447.0], ['ยากลำบาก', 448.0], ['สันติสุข', 449.0], ['หอม', 450.0], ['เต็ม', 451.0], ['เวียนหัว', 452.0], ['ร่วง', 453.0], ['เพราะว่า', 454.0], ['เอ็นดู', 45

In [12]:
#เปลี่ยน List ที่ได้ให้เป็น Dict เตรียมสำหรับ list of keyword

new_dict = dict(link_list_1)

print(new_dict)

, 'แบล็กลิสต์': 274.0, 'ป่าเถื่อน': 275.0, 'งอแง': 276.0, 'ไถ่โทษ': 277.0, 'เดียว': 278.0, 'บกพร่อง': 279.0, 'หยิ่ง': 280.0, 'ศัก': 281.0, 'เร็ว': 282.0, 'ช็อค': 283.0, 'เกือบ': 284.0, 'เจริญเติบโต': 285.0, 'ปรองดอง': 286.0, 'หายหน้า': 287.0, 'ค่าไถ่': 288.0, 'ยึดเหนี่ยว': 289.0, 'ชื่นชม': 290.0, 'ร้ายกาจ': 291.0, 'จอมปลอม': 292.0, 'ธรรมชาติ': 293.0, 'ส': 294.0, 'แทง': 295.0, 'อดอยาก': 296.0, 'ณ': 297.0, 'แฮก': 298.0, 'เห็น': 299.0, 'สารพัด': 300.0, 'แค': 301.0, 'ไร้': 302.0, 'เผด็จการ': 303.0, 'สิ่ง': 304.0, 'แปลกหน้า': 305.0, 'พลัดพราก': 306.0, 'เรียกร้อง': 307.0, 'แรงใจ': 308.0, 'ความสุข': 309.0, 'มั่ว': 310.0, 'พนัน': 311.0, 'แฮปปี้': 312.0, 'ดีขึ้น': 313.0, 'ขอบใจ': 314.0, 'อนาจาร': 315.0, 'ปลอดภัย': 316.0, 'เพชฌฆาต': 317.0, 'ฮา': 318.0, 'สาบสูญ': 319.0, 'พระศาสนา': 320.0, 'ขู่': 321.0, 'ให้โอกาส': 322.0, 'ถ่าย': 323.0, 'ถ้า': 324.0, 'เตก': 325.0, 'วะ': 326.0, 'ในหลวง': 327.0, 'ทำร้าย': 328.0, 'เดี่ยว': 329.0, 'เตือนสติ': 330.0, 'กระชาก': 331.0, 'ผ่อนคลาย': 332.0, 'ยกเลิก': 333.0,

In [13]:
#เริ่มทำการเปลี่ยนคำเป็น Tensor

new_list_2 = []

for a_list in new_word_list:
    list_of_word_num = []
    for a in a_list[0]:
        b = new_dict[a]
        list_of_word_num.append(b)
    list_of_word_tensor = torch.LongTensor(list_of_word_num)
    list_combine = []
    list_combine.append(a_list[1])
    list_combine.append(list_of_word_tensor)
    new_list_2.append(list_combine)

print(new_list_2)

or([132])], [2, tensor([824])], [3, tensor([1370])], [3, tensor([1108])], [3, tensor([424])], [3, tensor([485])], [3, tensor([811])], [3, tensor([1103])], [3, tensor([1451])], [3, tensor([240])], [3, tensor([1396])], [3, tensor([957])], [3, tensor([510])], [3, tensor([528])], [3, tensor([1406])], [3, tensor([1316])], [3, tensor([149])], [3, tensor([382])], [3, tensor([999])], [3, tensor([1041])], [3, tensor([1365])], [3, tensor([779])], [3, tensor([364])], [3, tensor([685])], [3, tensor([1367])], [3, tensor([1080])], [3, tensor([211])], [3, tensor([731])], [3, tensor([457])], [3, tensor([1166])], [3, tensor([869])], [3, tensor([164])], [3, tensor([500])], [3, tensor([927])], [3, tensor([117])], [3, tensor([978])], [3, tensor([1084])], [3, tensor([481])], [3, tensor([20])], [3, tensor([527])], [3, tensor([1318])], [3, tensor([1150])], [3, tensor([1300])], [3, tensor([644])], [3, tensor([475])], [3, tensor([324])], [3, tensor([1360,   86])], [3, tensor([913])], [3, tensor([753])], [3, te

In [14]:
#ทำการ Split Train, Test Data

train_data, test_data = train_test_split(new_list_2, test_size=0.3, random_state=42)

test_data[15]

[4, tensor([1447])]

In [15]:
#Define Model
#Reference: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [16]:
# Set hyperparemeter
VOCAB_SIZE = 200000
EMBED_DIM = 32
NUN_CLASS = len(new_list_2)
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)
BATCH_SIZE = 16


In [17]:
#Define Batch size

def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [18]:
#เขียน Function สำหรับ Train และ Test

from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text, offsets, cls
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text, offsets, cls
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [19]:
#Train Model 50 รอบ ผลยัง Overfit อยู่

import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 50
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_data) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_data, [train_len, len(train_data) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 0 seconds
	Loss: 0.1105(train)	|	Acc: 48.3%(train)
	Loss: 0.0942(valid)	|	Acc: 44.2%(valid)
Epoch: 2  | time in 0 minutes, 0 seconds
	Loss: 0.0318(train)	|	Acc: 83.9%(train)
	Loss: 0.0807(valid)	|	Acc: 55.8%(valid)
Epoch: 3  | time in 0 minutes, 0 seconds
	Loss: 0.0140(train)	|	Acc: 95.3%(train)
	Loss: 0.0731(valid)	|	Acc: 48.8%(valid)
Epoch: 4  | time in 0 minutes, 0 seconds
	Loss: 0.0092(train)	|	Acc: 97.5%(train)
	Loss: 0.0754(valid)	|	Acc: 55.8%(valid)
Epoch: 5  | time in 0 minutes, 0 seconds
	Loss: 0.0082(train)	|	Acc: 97.8%(train)
	Loss: 0.0754(valid)	|	Acc: 53.5%(valid)
Epoch: 6  | time in 0 minutes, 0 seconds
	Loss: 0.0061(train)	|	Acc: 98.0%(train)
	Loss: 0.0720(valid)	|	Acc: 53.5%(valid)
Epoch: 7  | time in 0 minutes, 0 seconds
	Loss: 0.0059(train)	|	Acc: 98.5%(train)
	Loss: 0.0753(valid)	|	Acc: 51.2%(valid)
Epoch: 8  | time in 0 minutes, 0 seconds
	Loss: 0.0045(train)	|	Acc: 98.5%(train)
	Loss: 0.0755(valid)	|	Acc: 51.2%(valid)
Epoch: 9  | time

In [20]:
#ลอง Test dataset ผลยังไม่ดี Overfit

print('Checking the results of test dataset...')
test_loss, test_acc = test(test_data)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0205(test)	|	Acc: 60.0%(test)


In [21]:
label_dataset = {
    1 : "Potential",
    2 : "High Potential",
    3 : "Natural",
    4 : "Negative",
    5 : "High Negative",
}

In [22]:
# ถ้าจะลองกับข้อมูลอื่นต้องแปลงให้มันเป็น Tensor ก่อนที่จะเอาไปใส่ Model

text = "น่าสนใจมาก"

text_tokenize = word_tokenize(text)
text_list = []
for word in text_tokenize:
    number = new_dict[word]
    text_list.append(number)
text_tensor = torch.LongTensor(text_list)

print(text_tensor)

tensor([1278, 1076])


In [23]:
#รวม แปลงเป็น Tensor และใส่ Model ไว้ใน Function เดียวกัน

def predict(text):
    text_tokenize = word_tokenize(text)
    text_list = []
    for word in text_tokenize:
        number = new_dict[word]
        text_list.append(number)
    text_tensor = torch.LongTensor(text_list)
    result = model(text_tensor,torch.tensor([0]))

    return label_dataset[result.argmax(1).item()]